# Initialization

In [2]:
%load_ext autoreload
%load_ext line_profiler
import sys
sys.path.append("..")

In [7]:
%autoreload

import numpy as np
import pickle

import main


import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc_file('matplotlibrc')

%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [10,10]


import darkhistory.physics as phys
import darkhistory.utilities as utils
import darkhistory.spec.transferfunction as tf
import darkhistory.spec.spectools as spectools

from darkhistory.spec.spectrum import Spectrum

from darkhistory.electrons.ics.ics_spectrum import ics_spec
from darkhistory.electrons.ics.ics_engloss_spectrum import engloss_spec
from darkhistory.electrons.elec_cooling import get_elec_cooling_tf_fast

from darkhistory.electrons import positronium

from scipy.interpolate import interp1d


In [4]:
ics_thomson_ref_tf=pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/tflists/ics/ics_thomson_ref_tf.raw", "rb")) 
ics_rel_ref_tf=pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/tflists/ics/ics_rel_ref_tf.raw", "rb"))
engloss_ref_tf=pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/tflists/ics/engloss_ref_tf.raw", "rb"))

In [9]:
%autoreload

eleceng = 10**np.arange(0, np.log10(5e12), 0.0254)
photeng = 10**np.arange(-4, np.log10(5e12), 0.0334)


rs = 600
xe = 1e-2

from darkhistory.spec.spectra import Spectra

coll_ion_sec_elec_specs = (
    phys.coll_ion_sec_elec_spec(eleceng, eleceng, species='HI'),
    phys.coll_ion_sec_elec_spec(eleceng, eleceng, species='HeI'),
    phys.coll_ion_sec_elec_spec(eleceng, eleceng, species='HeII')
)

coll_exc_sec_elec_tf_HI = tf.TransFuncAtRedshift(
    np.squeeze(np.identity(eleceng.size)[:, np.where(eleceng > phys.lya_eng)]),
    in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
    eng = eleceng[eleceng > phys.lya_eng] - phys.lya_eng,
    dlnz = -1, spec_type = 'N'
)

coll_exc_sec_elec_tf_HeI = tf.TransFuncAtRedshift(
    np.squeeze(np.identity(eleceng.size)[:, np.where(eleceng > phys.He_exc_eng)]),
    in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
    eng = eleceng[eleceng > phys.He_exc_eng] - phys.He_exc_eng,
    dlnz = -1, spec_type = 'N'
)

coll_exc_sec_elec_tf_HeII = tf.TransFuncAtRedshift(
    np.squeeze(np.identity(eleceng.size)[:, np.where(eleceng > 4*phys.lya_eng)]),
    in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
    eng = eleceng[eleceng > 4*phys.lya_eng] - 4*phys.lya_eng,
    dlnz = -1, spec_type = 'N'
)

coll_exc_sec_elec_tf_HI.rebin(eleceng)
coll_exc_sec_elec_tf_HeI.rebin(eleceng)
coll_exc_sec_elec_tf_HeII.rebin(eleceng)

coll_exc_sec_elec_specs = (
    coll_exc_sec_elec_tf_HI.grid_vals,
    coll_exc_sec_elec_tf_HeI.grid_vals,
    coll_exc_sec_elec_tf_HeII.grid_vals
)

from darkhistory.spec.spectools import EnglossRebinData

ics_engloss_data = EnglossRebinData(eleceng, photeng, eleceng)


../darkhistory/spec/spectra.py:401: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  np.issubdtype(type(other), float)
../darkhistory/spec/spectra.py:402: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  or np.issubdtype(type(other), int)


In [12]:
%%prun
get_elec_cooling_tf_fast(
    ics_thomson_ref_tf, ics_rel_ref_tf, engloss_ref_tf, 
    coll_ion_sec_elec_specs, coll_exc_sec_elec_specs, 
    eleceng, photeng, rs, xe, xHe=0, ics_engloss_data = ics_engloss_data, check_conservation_eng=True, verbose=True
)

***************************************************
rs:  600
injected energy:  3018.5613014197993
Energy in low energy electrons:  2901.50338868706
Energy in photons:  2672.465358972423
Continuum_engloss:  2630.84103571871
Energy in photons - Continuum:  41.6243232537131
Deposited in ionization:  40.13101693341628
Deposited in excitation:  27.84907658465386
Deposited in heating:  7.512255561388976
Energy is conserved up to (%):  -0.0019466094793372003
Deposited in ICS:  -0.0587596004339604
Energy conservation with deposited (%):  3.0419024154926075e-14
***************************************************
***************************************************
rs:  600
injected energy:  3200.368594586517
Energy in low energy electrons:  2963.516543788905
Energy in photons:  5388.308064862846
Continuum_engloss:  5301.769895970508
Energy in photons - Continuum:  86.53816889233804
Deposited in ionization:  79.94052174076654
Deposited in excitation:  55.56557393289815
Deposited in heating:  14.9

***************************************************
rs:  600
injected energy:  15523.870099580816
Energy in low energy electrons:  5777.955079438627
Energy in photons:  152597.41408139793
Continuum_engloss:  145765.4795593823
Energy in photons - Continuum:  6831.934522015625
Deposited in ionization:  1533.3108045292563
Deposited in excitation:  1112.8802241969859
Deposited in heating:  270.97751017301516
Energy is conserved up to (%):  -0.020536378829792975
Deposited in ICS:  -3.1880407726651727
Energy conservation with deposited (%):  -1.913398572617105e-13
***************************************************
***************************************************
rs:  600
injected energy:  16458.869432193034
Energy in low energy electrons:  5903.347844965153
Energy in photons:  161247.54561412503
Continuum_engloss:  153723.9152479481
Energy in photons - Continuum:  7523.63036617692
Deposited in ionization:  1594.7024622042202
Deposited in excitation:  1159.0763351325181
Deposited in heati

***************************************************
rs:  600
injected energy:  59593.65192559373
Energy in low energy electrons:  8339.451977514887
Energy in photons:  403220.18475829286
Continuum_engloss:  357280.7512242204
Energy in photons - Continuum:  45939.43353407248
Deposited in ionization:  2771.8686082527506
Deposited in excitation:  2066.1702540407746
Deposited in heating:  484.5257935895813
Energy is conserved up to (%):  -0.013085692225215563
Deposited in ICS:  -7.798241876709429
Energy conservation with deposited (%):  -6.714658533212734e-14
***************************************************
***************************************************
rs:  600
injected energy:  63182.96466918977
Energy in low energy electrons:  8426.856649437777
Energy in photons:  416535.7347702891
Continuum_engloss:  367176.6282860402
Energy in photons - Continuum:  49359.106484248885
Deposited in ionization:  2813.744860951295
Deposited in excitation:  2099.2340320676763
Deposited in heating: 

***************************************************
rs:  600
injected energy:  324937.6236187617
Energy in low energy electrons:  9964.297989016006
Energy in photons:  937316.2988942444
Continuum_engloss:  629212.9064248805
Energy in photons - Continuum:  308103.39246936387
Deposited in ionization:  3548.249053326432
Deposited in excitation:  2691.2352876107198
Deposited in heating:  644.1706344209792
Energy is conserved up to (%):  -0.004222907407106919
Deposited in ICS:  -13.721814975917882
Energy conservation with deposited (%):  -1.0956212210934796e-13
***************************************************
***************************************************
rs:  600
injected energy:  344508.54628657864
Energy in low energy electrons:  9991.360903842746
Energy in photons:  965015.3120177609
Continuum_engloss:  637395.150565443
Energy in photons - Continuum:  327620.16145231796
Deposited in ionization:  3561.151366257054
Deposited in excitation:  2701.912795155815
Deposited in heating: 

***************************************************
rs:  600
injected energy:  828323.5329759169
Energy in low energy electrons:  10248.470944593233
Energy in photons:  1554599.3255726495
Continuum_engloss:  743688.3971198401
Energy in photons - Continuum:  810910.9284528093
Deposited in ionization:  3683.696937155338
Deposited in excitation:  2804.271460716845
Deposited in heating:  692.3868689906361
Energy is conserved up to (%):  -0.001958375888504044
Deposited in ICS:  -16.221688348031375
Energy conservation with deposited (%):  -6.927530516937567e-14
***************************************************
***************************************************
rs:  600
injected energy:  878213.2798980062
Energy in low energy electrons:  10257.939132552801
Energy in photons:  1610345.3571984696
Continuum_engloss:  749564.6638792023
Energy in photons - Continuum:  860780.6933192672
Deposited in ionization:  3688.2086859145033
Deposited in excitation:  2808.085030528699
Deposited in heating:

...

***************************************************
rs:  600
injected energy:  614327561.2705201
Energy in low energy electrons:  10362.299868001599
Energy in photons:  615182243.0525941
Continuum_engloss:  872362.3431095603
Energy in photons - Continuum:  614309880.7094845
Deposited in ionization:  3737.9342968987157
Deposited in excitation:  2850.649979601287
Deposited in heating:  748.7100017560125
Energy is conserved up to (%):  -3.098202296377083e-06
Deposited in ICS:  -19.033110784253793
Energy conservation with deposited (%):  2.8273712608584176e-14
***************************************************
***************************************************
rs:  600
injected energy:  651328377.1823176
Energy in low energy electrons:  10362.299925608546
Energy in photons:  652183058.9571643
Continuum_engloss:  872362.3432637269
Energy in photons - Continuum:  651310696.6139005
Deposited in ionization:  3737.934324353284
Deposited in excitation:  2850.6500050881236
Deposited in heating:

***************************************************
rs:  600
injected energy:  5670221181.605416
Energy in low energy electrons:  10362.3003977677
Energy in photons:  5671075863.271287
Continuum_engloss:  872362.3445275157
Energy in photons - Continuum:  5670203500.92676
Deposited in ionization:  3737.9345493824458
Deposited in excitation:  2850.6502149694184
Deposited in heating:  748.8270811559896
Energy is conserved up to (%):  -3.356762671703405e-07
Deposited in ICS:  -19.033588094776743
Energy conservation with deposited (%):  2.27867955414962e-14
***************************************************
***************************************************
rs:  600
injected energy:  6011737374.832779
Energy in low energy electrons:  10362.300398562631
Energy in photons:  6012592056.497774
Continuum_engloss:  872362.3445296438
Energy in photons - Continuum:  6011719694.153244
Deposited in ionization:  3737.9345497613385
Deposited in excitation:  2850.6502153279116
Deposited in heating:  7

***************************************************
***************************************************
rs:  600
injected energy:  66130224948.17659
Energy in low energy electrons:  10362.300405345959
Energy in photons:  66131079629.827255
Continuum_engloss:  872362.3445478097
Energy in photons - Continuum:  66130207267.482704
Deposited in ionization:  3737.9345529945645
Deposited in excitation:  2850.6502183994735
Deposited in heating:  748.8427746296411
Energy is conserved up to (%):  -2.8782705102002198e-08
Deposited in ICS:  -19.03407907392565
Energy conservation with deposited (%):  1.7304948386306282e-14
***************************************************
***************************************************
rs:  600
injected energy:  70113234068.67345
Energy in low energy electrons:  10362.300405354896
Energy in photons:  70114088750.32396
Continuum_engloss:  872362.3445478337
Energy in photons - Continuum:  70113216387.97942
Deposited in ionization:  3737.934552998824
Deposited i

***************************************************
rs:  600
injected energy:  483058802039.77185
Energy in low energy electrons:  10362.300405438302
Energy in photons:  483059656721.42145
Continuum_engloss:  872362.3445480575
Energy in photons - Continuum:  483058784359.0769
Deposited in ionization:  3737.9345530385826
Deposited in excitation:  2850.6502184420174
Deposited in heating:  748.8454349349903
Energy is conserved up to (%):  -3.940653826244239e-09
Deposited in ICS:  -19.035529915564084
Energy conservation with deposited (%):  -3.00688084253316e-14
***************************************************
***************************************************
rs:  600
injected energy:  512153329024.49744
Energy in low energy electrons:  10362.30040543864
Energy in photons:  512154183706.1467
Continuum_engloss:  872362.3445480586
Energy in photons - Continuum:  512153311343.8022
Deposited in ionization:  3737.934553038744
Deposited in excitation:  2850.6502184421743
Deposited in heatin

***************************************************
rs:  600
injected energy:  1095468767333.9965
Energy in low energy electrons:  10362.300405441067
Energy in photons:  1095469622015.645
Continuum_engloss:  872362.3445480649
Energy in photons - Continuum:  1095468749653.3004
Deposited in ionization:  3737.934553039901
Deposited in excitation:  2850.650218443311
Deposited in heating:  748.8461756600847
Energy is conserved up to (%):  -1.7376412571587802e-09
Deposited in ICS:  -19.035258923002925
Energy conservation with deposited (%):  -5.325344095921868e-15
***************************************************
***************************************************
rs:  600
injected energy:  1161448613840.3413
Energy in low energy electrons:  10362.300405441169
Energy in photons:  1161449468521.9907
Continuum_engloss:  872362.3445480654
Energy in photons - Continuum:  1161448596159.6462
Deposited in ionization:  3737.934553039949
Deposited in excitation:  2850.6502184433584
Deposited in hea